<a href="https://colab.research.google.com/github/duongtrung/Pytorch-tutorials/blob/main/11_AWS_forecast_quick_start_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AWS forecast quick start

AWS forecast involves the following 3 steps:

<img src="resources/11-aws-quick-start-forecasting-overview.png" width=900 alt="a aws forecast workflow"/>

Imagine we are trying to solve the forecasting problem for a ride-hailing service and we want to predict how many pick-ups are expected in specific areas of New York. For this exercise, we will use the yellow taxi trip records from [NYC Taxi and Limousine Commission (TLC)](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

We will start by importing the historical data from December 2017 to January 2019, [link](https://raw.githubusercontent.com/aws-samples/amazon-forecast-samples/main/notebooks/basic/Getting_Started/data/taxi-dec2017-jan2019.csv). Next, we will train a Predictor using this data. Finally, we will generate a forecast for February 2019 and compare it with the actual data from February 2019, [link](https://raw.githubusercontent.com/aws-samples/amazon-forecast-samples/main/notebooks/basic/Getting_Started/data/taxi-feb2019.csv).

You need to download the experimental datasets and put them in your data folder.

### Pre-requisites

Before we get started, lets set up the notebook environment, the AWS SDK client for Amazon Forecast and IAM Role used by Amazon Forecast to access your data.


In [1]:
# The following installation commmands are used for anaconda windows 11 64bit
# I recommend to create conda env for your experiments
# If you use other OS or docker, google how to prepare your machine.

# conda install -c conda-forge s3fs
# conda install -c anaconda ipywidgets
# conda install -c anaconda boto3
# conda install -c conda-forge awscli

In [2]:
import sys
import os

sys.path.insert( 0, os.path.abspath("common-aws-forecast") )

import json
import util
import boto3
import s3fs
import pandas as pd

### Create an instance of AWS SDK client

In [4]:
region = 'eu-central-1' # Europe (Frankfurt)
session = boto3.Session(region_name=region) 
forecast = session.client(service_name='forecast')
forecastquery = session.client(service_name='forecastquery')

# Checking to make sure we can communicate with Amazon Forecast
assert forecast.list_predictors()

NoCredentialsError: Unable to locate credentials